# We match our rating data to the main dataset

## Importing libraries

In [68]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import date

## Loading data

In [69]:
ratings_df = pd.read_csv('Prepared Frames/rating_data.csv')
df_companies = pd.read_csv('Prepared Frames/companies_approx_match.csv')
main_df = pd.read_csv('Prepared Frames/main_data.csv')

## Merging the found matches to the main Dataframe

In [70]:
main_df = pd.merge(left = main_df,
                                 right = df_companies[['Ticker','difflib']],
                                 # We use outer to not loose Information, we will have to get rid of some NaNs later
                                 how = 'left', 
                                 left_on = ['Ticker'], 
                                 right_on= ['Ticker'])

## Assigning the ratings
We now have to assign the ratings. The difficulty is, that we don't have the exact dates to match, but have to match when the date in the main dataframe is greater than the one for the rating and have to overwrite it with a new rating in case there is one.

In [71]:
# We can only assign ratings where we have found the companies, 
# we create a new dataframe just with these entries
df_rated = main_df[main_df['difflib'].isna()==False].copy()

In [72]:
# We sort the dataframe by dates
df_rated = df_rated.sort_values(by = ['Report Date'], ascending = True)
df_rated.head(3)

,Ticker,Company Name,IndustryId,Fiscal Year,Fiscal Period,Report Date,Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Total Current Assets,...,Change in Fixed Assets & Intangibles,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash,Sector,Industry,difflib
18103,KEYS,"Keysight Technologies, Inc.",101001.0,2017,Q1,2017-01-31,173000000.0,8.960000e+08,395000000.0,1.932000e+09,...,-8000000.0,-62800000.0,-8000000.0,5.369231e+07,19000000.0,21000000.0,113000000,Technology,Computer Hardware,"keysight technologies, inc."
17887,JWN,NORDSTROM INC,103002.0,2016,Q4,2017-01-31,178500000.0,1.007000e+09,199000000.0,3.242000e+09,...,-221000000.0,0.0,-213000000.0,-3.000000e+06,-154000000.0,-91000000.0,476000000,Consumer Cyclical,Retail - Apparel & Specialty,"nordstrom, inc."
5334,BURL,"Burlington Stores, Inc.",102001.0,2016,Q4,2017-01-31,70877000.0,8.159700e+07,43252000.0,9.283240e+08,...,-49864000.0,0.0,-42812000.0,-1.743000e+08,-50025000.0,-226086000.0,48798000,Consumer Defensive,Retail - Defensive,"burlington stores, inc."


In [73]:
# Since we matched the names of the companies on the lowercase version, we have to create a column  with that version again
ratings_df['Company_lower'] = ratings_df['Company'].apply(lambda x: x.lower())

In [74]:
# We create a list of all unique companies in our ratings dataframe
companies = list(df_rated['difflib'].unique())

#### Checking if we can match right

In [75]:
df_rated[df_rated['difflib']==companies[0]].head(2)

,Ticker,Company Name,IndustryId,Fiscal Year,Fiscal Period,Report Date,Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Total Current Assets,...,Change in Fixed Assets & Intangibles,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash,Sector,Industry,difflib
18103,KEYS,"Keysight Technologies, Inc.",101001.0,2017,Q1,2017-01-31,173000000.0,8.960000e+08,395000000.0,1.932000e+09,...,-8000000.0,-62800000.0,-8.000000e+06,5.369231e+07,19000000.0,2.100000e+07,113000000,Technology,Computer Hardware,"keysight technologies, inc."
18104,KEYS,"Keysight Technologies, Inc.",101001.0,2017,Q2,2017-04-30,179000000.0,1.023000e+09,518000000.0,2.314000e+09,...,-17000000.0,-62800000.0,-1.634000e+09,1.239000e+09,446000000.0,1.656000e+09,87000000,Technology,Computer Hardware,"keysight technologies, inc."


In [76]:
ratings_df[ratings_df['Company_lower']==companies[0]]

,Company,Date,Rating,Company_lower
49310,"Keysight Technologies, Inc.",2014-09-12,Baa3,"keysight technologies, inc."
49311,"Keysight Technologies, Inc.",2019-08-14,Baa2,"keysight technologies, inc."


In [77]:
ratings_df.columns

Index(['Company', 'Date', 'Rating', 'Company_lower'], dtype='object')

#### We have to make sure to overwrite only the greater dates with updated rating information
We will test it manually for the first company and do the rest with a function

In [78]:
# First create a sub-dataframe only with the ratings we are interested in
current_ratings = ratings_df[ratings_df['Company_lower']==companies[0]].sort_values(by = ['Date'], ascending = True).reset_index(drop = True)
current_ratings

,Company,Date,Rating,Company_lower
0,"Keysight Technologies, Inc.",2014-09-12,Baa3,"keysight technologies, inc."
1,"Keysight Technologies, Inc.",2019-08-14,Baa2,"keysight technologies, inc."


In [79]:
for action in range(len(current_ratings)):
    # Getting the needed information
    company = current_ratings.loc[action,'Company_lower']
    date = current_ratings.loc[action,'Date']
    rating = current_ratings.loc[action,'Rating']
    
    # We need to adress the part of the dataframe where the company is the same and the date higher
    df_rated.loc[(df_rated['difflib']==company)&(df_rated['Report Date']>=date),'Rating'] = rating

In [81]:
# Checking the result
# df_rated[df_rated['difflib']==companies[0]]

# That worked correctly, we can apply it to our whole dataframe

In [82]:
def assign_ratings():
    # List of companies 
    companies = list(df_rated['difflib'].unique())
    for company in companies:
        # Creating the subframe
        current_ratings = ratings_df[ratings_df['Company_lower']==company].sort_values(by = ['Date'], ascending = True).reset_index(drop = True)
        for action in range(len(current_ratings)):
            # Getting the needed information
            company = current_ratings.loc[action,'Company_lower']
            date = current_ratings.loc[action,'Date']
            rating = current_ratings.loc[action,'Rating']
             # We need to adress the part of the dataframe where the company is the same and the date higher
            df_rated.loc[(df_rated['difflib']==company)&(df_rated['Report Date']>=date),'Rating'] = rating

In [83]:
# We apply the ratings
assign_ratings()

In [84]:
# Checking results
df_rated.isna().sum()

Ticker                                                0
Company Name                                          0
IndustryId                                            0
Fiscal Year                                           0
Fiscal Period                                         0
Report Date                                           0
Shares (Diluted)                                      0
Cash, Cash Equivalents & Short Term Investments       0
Accounts & Notes Receivable                           0
Total Current Assets                                  0
Property, Plant & Equipment, Net                      0
Other Long Term Assets                                0
Total Noncurrent Assets                               0
Total Assets                                          0
Payables & Accruals                                   0
Total Current Liabilities                             0
Total Noncurrent Liabilities                          0
Total Liabilities                               

In [85]:
# We got almost everything rated
df_rated[df_rated['Rating'].isna()]

,Ticker,Company Name,IndustryId,Fiscal Year,Fiscal Period,Report Date,Shares (Diluted),"Cash, Cash Equivalents & Short Term Investments",Accounts & Notes Receivable,Total Current Assets,...,Net Cash from Acquisitions & Divestitures,Net Cash from Investing Activities,Cash from (Repayment of) Debt,Cash from (Repurchase of) Equity,Net Cash from Financing Activities,Net Change in Cash,Sector,Industry,difflib,Rating
18252,KMG,KMG CHEMICALS INC,110001.0,2017,Q2,2017-01-31,12293000.0,3.058700e+07,35309000.0,1.124910e+08,...,-1.280460e+08,-2676000.0,7.700000e+06,43909250.0,7.124000e+06,12981000,Basic Materials,Chemicals,"kmg chemicals, inc.",NaN
7091,CLDR,"Cloudera, Inc.",101003.0,2016,Q4,2017-01-31,36848463.0,2.349560e+08,101549000.0,3.497020e+08,...,0.000000e+00,75626000.0,0.000000e+00,-1015000.0,-1.015000e+06,42738000,Technology,Application Software,"cloudera, inc",NaN
21862,MRVL,MARVELL TECHNOLOGY GROUP LTD,101004.0,2016,Q4,2017-01-31,520623000.0,1.668360e+09,335384000.0,2.290434e+09,...,0.000000e+00,44257000.0,4.189121e+07,-125033000.0,-1.005990e+08,62504000,Technology,Semiconductors,marvell technology group ltd.,NaN
33972,UNFI,UNITED NATURAL FOODS INC,102001.0,2017,Q2,2017-01-31,50755000.0,3.065800e+07,514870000.0,1.638398e+09,...,9.200000e+04,-15366000.0,-7.193300e+07,165000.0,-7.179100e+07,17102000,Consumer Defensive,Retail - Defensive,"united natural foods, inc",NaN
20063,M,"Macy's, Inc.",103002.0,2016,Q4,2017-01-31,307800000.0,1.297000e+09,522000000.0,7.626000e+09,...,0.000000e+00,304000000.0,-6.280000e+08,-81000000.0,-9.570000e+08,840000000,Consumer Cyclical,Retail - Apparel & Specialty,"macy's, inc.",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7796,COIN,"Coinbase Global, Inc.",104003.0,2021,Q2,2021-06-30,126996000.0,4.365982e+09,184579000.0,1.423806e+10,...,-1.646700e+07,-338693000.0,1.465034e+09,90478000.0,1.413711e+09,5052701000,Financial Services,"Brokers, Exchanges & Other","coinbase global, inc.",NaN
15701,HQY,HEALTHEQUITY INC,101003.0,2021,Q2,2021-07-31,83509000.0,7.537540e+08,74223000.0,8.606140e+08,...,-2.344000e+06,-20467000.0,0.000000e+00,2483000.0,2.000000e+05,16981000,Technology,Application Software,"healthequity, inc.",NaN
26520,PRO,"PROS Holdings, Inc.",101003.0,2021,Q3,2021-09-30,44318000.0,3.086420e+08,43125000.0,3.681300e+08,...,-2.482440e+07,-2516000.0,-2.880000e+05,1515000.0,1.227000e+06,-9684000,Technology,Application Software,"prog holdings, inc.",NaN
26418,PRG,"PROG Holdings, Inc.",104007.0,2021,Q3,2021-09-30,66385000.0,1.287880e+08,180231000.0,8.977520e+08,...,-1.930000e+05,-13848000.0,-4.123300e+07,-50760000.0,-5.096200e+07,-8761000,Financial Services,Credit Services,"prog holdings, inc.",NaN


In [86]:
ratings_df[ratings_df['Company_lower']=="prog holdings, inc."]

,Company,Date,Rating,Company_lower
59715,"PROG HOLDINGS, INC.",2021-11-08,B1,"prog holdings, inc."


I looked through a few of them and realized, that the NaNs are created for the quarters where there is no rating available yet for the companies, so we drop those lines.

In [87]:
df_rated.shape

(7086, 37)

In [88]:
df_rated = df_rated.dropna(axis='rows')

In [89]:
df_rated.shape

(6079, 37)

## Saving our finalized dataframe

In [90]:
df_rated.to_csv('Prepared Frames/rated.csv', index=False)